In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2", temperature=0.5)
response = llm.invoke("Tell me a joke about programming.")

AIMessage(content='Why do programmers prefer dark mode?\n\nBecause light attracts bugs.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2026-01-24T07:35:11.8492167Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6318894400, 'load_duration': 432035300, 'prompt_eval_count': 32, 'prompt_eval_duration': 3313849700, 'eval_count': 13, 'eval_duration': 2446246200, 'logprobs': None, 'model_name': 'llama3.2', 'model_provider': 'ollama'}, id='lc_run--019beeed-9203-7ef2-ace3-b34959de3f02-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 32, 'output_tokens': 13, 'total_tokens': 45})

In [4]:
response.content

'Why do programmers prefer dark mode?\n\nBecause light attracts bugs.'

In [31]:
from typing import Optional
import re, time
import requests
from bs4 import BeautifulSoup
from pydantic import BaseModel, HttpUrl, PositiveInt, ValidationError
from cachetools import TTLCache, cached

# class is from pydantic to validate input parameters
class UrlSummarizeInput(BaseModel):
    url: HttpUrl
    max_length: PositiveInt = 120

# function to clean text by removing extra spaces and new lines
def clean_text(txt):
    txt = re.sub(r'\s+',' ', txt).strip()
    return txt

# function to extract main text from HTML content, beautifulsoup is used for parsing HTML which concerts html content to readable text and we are focusing on <article> tags or all <p> tags
def extract_main_text(html):
    soup = BeautifulSoup(html, 'html.parser')
    article = soup.find('article')
    if article:
        text = " ".join(p.get_text(" ", strip=True) for p in article.find_all('p'))
    else:
        text = " ".join(p.get_text(" ", strip=True) for p in soup.find_all('p'))
    return clean_text(text)

# using the url to fetch the content with retries and backoff strategy
def http_get(url, retries = 3, bacoff = 1.5, timeout = 15):
    last_err = None
    for attempt in range(1, retries + 1):
        try:
            r = requests.get(url, timeout=timeout, headers = {'User-Agent': 'AgenticAI/1.0'})
            r.raise_for_status()
            return r
        except requests.RequestException as e:
            last_err = e
            if attempt == retries:
                raise
            time.sleep(bacoff**attempt)
    raise last_err


# caching the fetched and extracted content for 10 minutes to avoid redundant network calls, tt stands for time to live there are other caching techniques like LFU, LRU etc. last recetly used, last frequently used
cache = TTLCache(maxsize = 256, ttl = 600)

@cached(cache)
def fetch_and_extract(url):
    resp = http_get(url)
    return extract_main_text(resp.text)

# It is the main funcrtion that summarizes the content from the URL, it validates the input, fetches and extracts the text, and then returns words based on the max_words parameter.
## It basically gives at max 120 words from the content of the URL provided.
def summarize_url(url, max_words = 120):
    try:
        args = UrlSummarizeInput(url = url, max_length = max_words)
    except ValidationError as ve:
        return f"Input validation error: {ve}"
    
    try:
        text = fetch_and_extract(str(args.url))
        if not text or len(text.split()) < 30:
            return "Sorry, could not find enough readable content on that page." 
    except Exception as e:
        return f"Failed to fetch or parse the URL: {e}"
    
    words = text.split()
    if len(words) <= args.max_length:
        return " ".join(words)
    
    out = []
    count = 0
    for sentence in re.split(r'(?<=[.!?])\s+', text):
        sw = sentence.split()
        if count + len(sw) > args.max_length:
            break
        out.append(sentence)
        count += len(sw)

    if not out:
        out = words[:args.max_length]
        return " ".join(out) + "..."
    return " ".join(out)
    


In [32]:
summarize_url("https://en.wikipedia.org/wiki/Artificial_intelligence", max_words=120)

'Artificial intelligence ( AI ) is the capability of computational systems to perform tasks typically associated with human intelligence , such as learning , reasoning , problem-solving , perception , and decision-making . It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.'

In [ ]:
# Initializing custom tools using LangChain's tool decorator
## description should be clear about the input format and purpose of the tool
from langchain.tools import tool

@tool("Url_Summarizer", description="Fetches and summarizes a web page from a given URL. Input JSON with keys: 'url' (str) and optional 'max_words' (int, default 120). Use for extracting the main content and getting a concise summary.")
def summary_url(url: str, max_words: int = 120) -> str:
    return summarize_url(url, max_words)

@tool("Calculator", description="A simple calculator to perform basic arithmetic operations. Input should be a valid mathematical expression as a string, e.g., '2 + 2' or 'sqrt(16)'.")
def calculator(expr):
    try:
        return str(eval(expr))
    except Exception as e:
        return f"Error evaluating expression: {e}"




<class 'langchain_core.tools.structured.StructuredTool'>
<class 'function'>


In [57]:
# Using crate_agent to build an agent with the custom tools and the LLM. system prompt is provided to guide the agent's behavior.
from langchain.agents import create_agent

agent = create_agent(
    tools=[summary_url, calculator],
    model=llm,
    system_prompt=" You are an intelligent and helpful assistant. Use the available tools to answer user queries effectively.")

message = {"messages": [{"role": "user", "content": "Use UrlSummarizer with {'url':'https://en.wikipedia.org/wiki/Artificial_intelligence', 'max_words': 120}"}]}

result = agent.invoke(message)



In [58]:
# Given exact values and the tool is invoked successfully, the model is able to give the correct response.
result

{'messages': [HumanMessage(content="Use UrlSummarizer with {'url':'https://en.wikipedia.org/wiki/Artificial_intelligence', 'max_words': 120}", additional_kwargs={}, response_metadata={}, id='21750ac3-3a7a-4012-bed8-7c2781499c04'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2026-01-24T10:16:17.5433827Z', 'done': True, 'done_reason': 'stop', 'total_duration': 26885832400, 'load_duration': 588900300, 'prompt_eval_count': 300, 'prompt_eval_duration': 19702737600, 'eval_count': 37, 'eval_duration': 6302247700, 'logprobs': None, 'model_name': 'llama3.2', 'model_provider': 'ollama'}, id='lc_run--019bef80-be3e-79f3-b0a6-446d14e6e642-0', tool_calls=[{'name': 'Url_Summarizer', 'args': {'max_words': 120, 'url': 'https://en.wikipedia.org/wiki/Artificial_intelligence'}, 'id': '30987824-4e4a-4aed-9ebf-687e1b259ab6', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 300, 'output_tokens': 37, 'total_tokens': 337}),
 

In [59]:
# Using the UrlSummarizer tool to summarize a webpage with realistic query, the tool is invoked but the model is hallucinating during the response.
message = {"messages": [{"role": "user", "content": "Summarize this page in ~80 words using UrlSummarizer: {'url':'https://en.wikipedia.org/wiki/Artificial_intelligence', 'max_words': 120}?"}]}

result = agent.invoke(message)
result

{'messages': [HumanMessage(content="Summarize this page in ~80 words using UrlSummarizer: {'url':'https://en.wikipedia.org/wiki/Artificial_intelligence', 'max_words': 120}?", additional_kwargs={}, response_metadata={}, id='06ffcdf9-a492-409d-8aa7-cb6d1f32d1a0'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2026-01-24T10:17:13.0228143Z', 'done': True, 'done_reason': 'stop', 'total_duration': 15487103500, 'load_duration': 380724700, 'prompt_eval_count': 309, 'prompt_eval_duration': 11451704300, 'eval_count': 37, 'eval_duration': 3475806600, 'logprobs': None, 'model_name': 'llama3.2', 'model_provider': 'ollama'}, id='lc_run--019bef81-c3af-77e1-ba38-75f3adab6611-0', tool_calls=[{'name': 'Url_Summarizer', 'args': {'url': 'https://en.wikipedia.org/wiki/Artificial_intelligence', 'max_words': 80}, 'id': '5dea38c4-113c-4aa2-8aeb-0580ce1ea76c', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 309, 'output_tokens

In [60]:
# Calculator tool usage
message = {"messages": [{"role": "user", "content": "What is the value of 25*(8-3)?"}]}
result = agent.invoke(message)
result

{'messages': [HumanMessage(content='What is the value of 25*(8-3)?', additional_kwargs={}, response_metadata={}, id='3618abe3-bed1-4b9e-a61b-5050dffe8907'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2026-01-24T10:17:41.4316503Z', 'done': True, 'done_reason': 'stop', 'total_duration': 12650597700, 'load_duration': 376663100, 'prompt_eval_count': 284, 'prompt_eval_duration': 10246829200, 'eval_count': 21, 'eval_duration': 1949780200, 'logprobs': None, 'model_name': 'llama3.2', 'model_provider': 'ollama'}, id='lc_run--019bef82-3dca-7c50-b69a-3a52b2690031-0', tool_calls=[{'name': 'Calculator', 'args': {'expr': '25*(8-3)'}, 'id': '9fe52d0e-814b-49f3-9f76-c1fc4793cffb', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 284, 'output_tokens': 21, 'total_tokens': 305}),
  ToolMessage(content='125', name='Calculator', id='143ce816-39f5-48cf-889e-20e30c747107', tool_call_id='9fe52d0e-814b-49f3-9f76-c1fc4793cffb